# Using SQL Window Functions on an SQL Interview Question

*This SQL problem is a medium difficulty SQL interview problem on [datalemur.com](https://datalemur.com/questions/sql-third-transaction) for Uber that requires the use of ranking data with window functions to solve it.* 

*I'd heard about window functions, but had never needed to use it in a commercial setting. I went through a tutorial of SQL window functions over at [mode.com](https://mode.com/sql-tutorial/sql-window-functions), and then solved the problem.* 

*Not only does this demonstrate I know how to use window functions (and the fact they're pretty cool), but that I know how to learn something new to solve a problem. It only took me 2 hours to upskill and figure this out.* 

*A bit of house-keeping. This is an SQL problem. So to carry out the SQL problems in Jupyter, I made use of the `%sql` magic command with jupysql.* 

*Alright, enough pre-amble. Onto the queries!*

## The Problem
*Copied and pasted from datalemur.com*

This is the same question as problem #11 in the SQL Chapter of [Ace the Data Science Interview](https://amzn.to/3kF79Fx)!

Assume you are given the table below on Uber transactions made by users. Write a query to obtain the third transaction of every user. Output the user id, spend and transaction date.

### `transactions` Table:

|**Column Name**|**Type**|
|---|---|
|user_id|integer|
|spend|decimal|
|transaction_date|timestamp|

### `transactions` Example Input:

|**user_id**|**spend**|**transaction_date**|
|---|---|---|
|111|100.50|01/08/2022 12:00:00|
|111|55.00|01/10/2022 12:00:00|
|121|36.00|01/18/2022 12:00:00|
|145|24.99|01/26/2022 12:00:00|
|111|89.60|02/05/2022 12:00:00|

### Example Output:

|**user_id**|**spend**|**transaction_date**|
|---|---|---|
|111|89.60|02/05/2022 12:00:00|

The dataset you are querying against may have different input & output - **this is just an example**!

## Loading the data

In [1]:
#Load the SQL kernel
%load_ext sql

In [2]:
#Connect to database
%sql sqlite:///transactions.db

Connecting to 'sqlite:///transactions.db'

In [3]:
%%sql
--Inspect the data
SELECT * 
FROM transactions;

Running query in 'sqlite:///transactions.db'

user_id,spend,transaction_date
111,100.5,01/08/2022 12:00
111,55,01/10/2022 12:00
121,36,01/18/2022 12:00:00
145,24.99,01/26/2022 12:00:00
111,89.6,02/05/2022 12:00
145,45.3,02/28/2022 12:00:00
121,22.2,04/01/2022 12:00
121,67.9,04/03/2022 12:00
263,156,04/11/2022 12:00
230,78.3,06/14/2022 12:00:00


## Solution 

In [4]:
%%sql
SELECT 
  user_id,
  spend,
  transaction_date
FROM (
  SELECT 
    user_id,
    spend, 
    transaction_date,
    RANK() OVER (PARTITION BY user_id ORDER BY transaction_date) AS rank_id
  FROM transactions
) AS rank_transactions
WHERE rank_transactions.rank_id = 3;

Running query in 'sqlite:///transactions.db'

user_id,spend,transaction_date
111,89.6,02/05/2022 12:00
121,67.9,04/03/2022 12:00
263,100,07/12/2022 12:00


---

### Explanation of solution

We need to see the third transaction of every user, and we only want to see the user_id, the spend, and the transaction date. 

To get the third transaction of every user, we select the entire dataset, and then we carry out a **window rank** function which ranks the data by the transaction date, and segments it by the user_id. This is the 

```SQL
RANK() OVER (PARTITION BY user_id ORDER BY transaction_date) AS rank_id
```

part of the code. 

In [5]:
%%sql
SELECT 
  user_id,
  spend, 
  transaction_date,
  RANK() OVER (PARTITION BY user_id ORDER BY transaction_date) AS rank_id
FROM transactions

Running query in 'sqlite:///transactions.db'

user_id,spend,transaction_date,rank_id
None,None,None,1
None,None,None,1
111,100.5,01/08/2022 12:00,1
111,55,01/10/2022 12:00,2
111,89.6,02/05/2022 12:00,3
121,36,01/18/2022 12:00:00,1
121,22.2,04/01/2022 12:00,2
121,67.9,04/03/2022 12:00,3
145,24.99,01/26/2022 12:00:00,1
145,45.3,02/28/2022 12:00:00,2


Here we can see the rank for each transaction (in `rank_id`). To suppress the `rank_id` and to choose every third transaction, we wrap the query into a subquery (named as `rank_transactions`), and then we use a `WHERE` command at the end to filter for where `rank_id` is equal to 3. 

See. Easy. 